In [30]:
#Importing Libraries
import pandas as pd
import numpy as np
import glob
from datetime import date,timedelta
from nsepy import get_history
from pyfinance.ols import PandasRollingOLS
import statsmodels.api as sm
import talib as tb
import os

//anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [5]:
PATH = 'Data/'

In [22]:
def portfolioCreator(filter_name="",PATH = 'Data/'):
    """
    Creates a dictionary of dataframes(dates*stocks) of Close,
    Open,High,Low and Turnover
    """
    
    #Reading the dates and allocating memory
    dates = pd.read_csv(os.path.join(PATH,'dates.csv'),header = None,names = ["Date"])
    dates['Date'] = pd.to_datetime(dates['Date'],yearfirst = True)
    files = glob.glob(os.path.join(PATH,'nse_*.csv'))
    portfolio = dict()
    
    for file in files:
        
        df = pd.read_csv(file)
        df['_id']= dates['Date']
        df.set_index('_id',inplace = True)
        df.index.names = ['Date']
        
        #Creating the Universe
        
        if(filter_name == 'NIFTY'):
            ticker_list = pd.read_csv(os.path.join(PATH,'ind_nifty50list.csv'))['Symbol']
        elif(filter_name == 'FNO'):
            ticker_list = pd.read_csv(os.path.join(PATH,'fno_list.csv'))['Symbol']
        else:
            ticker_list = df.columns
            
        df = df.loc[:,ticker_list].dropna(axis=1, how='all')
        portfolio[file[file.find('.')+1:file.rfind('.')]] = df
        
    return portfolio

In [23]:
portfolio = portfolioCreator()

In [25]:
def createFilter(df = pd.DataFrame(),N = 200):
    """
    Creates a filter based on n-day Moving Average
    """
    
    _filter = pd.DataFrame(index = df.index,columns = df.columns)
    
    for stock in df.columns:
        _filter.loc[:,stock] = df[stock]>df[stock].rolling(N).mean()
    _filter.index = pd.to_datetime(_filter.index,yearfirst=True)
    
    return _filter

In [27]:
stock_filter = createFilter(portfolio['Close'],100)

In [48]:
def regressionRunner(portfolio = dict()):
    df = np.log(portfolio['Close'])
    df = df.reset_index()
    date_idx = df.Date
    df = df.drop(['Date'],axis=1)
    
    rsq = pd.DataFrame(index= df.index, columns=df.columns)
    beta = pd.DataFrame(index= df.index, columns=df.columns)
    
    for stock in df.columns:
        model = PandasRollingOLS(y=df[stock],x=df.index, window=90,has_const=False)
        rsq[stock]= model.rsq[model.rsq<1]
        beta[stock]=model.beta
        
    rsq.index = date_idx
    beta.index = date_idx
    return beta,rsq

In [49]:
beta,rsq = regressionRunner(portfolio)

In [51]:
def ATR(portfolio = dict()):
    
    atr_20 = pd.DataFrame(index= portfolio['High'].index, columns=portfolio['High'].columns)
    for stock in atr_20.columns:
        atr_20[stock]=tb.ATR(portfolio['High'][stock], portfolio['Low'][stock], portfolio['Close'][stock], timeperiod=20)
        
    return atr_20

In [52]:
ATR20 = ATR(portfolio)